### Busqueda de los vectores restantes para las bases (0,9,0).

**Resumen:** Utilizando los solvers de SageMath y de Mathematica obtengo un séptimo vector ortonormal a la primera base de 6 vectores con dos grados de libertad $a$ y $b$, pero al calcular los productos internos con la primera base completa, obtuve un producto igual a cero (con el primer vector) independientemente de los valores de $a$ y $b$, por lo que el candidato no sirve. Voy a investigar como funcionan los solvers y/o hacerlo a mano para ver si hay otra forma de solución.

In [1]:
%display latex

Usamos la extensión de Galois para construir la transformada de Fourier:

In [2]:
p = 2
n = 3
dim = p**n
FF = GF(dim, 'x')

def omega(x):
    return exp(I * 2 * pi * int(x.trace()) / p)

In [3]:
F = zero_matrix(SR, dim, dim)
for i, m in enumerate(FF):
    for j, n in enumerate(FF):
        F[i,j] = omega(m * n) / sqrt(dim)
F * sqrt(dim)

[ 1  1  1  1  1  1  1  1]
[ 1  1 -1  1 -1 -1 -1  1]
[ 1 -1  1 -1 -1 -1  1  1]
[ 1  1 -1 -1 -1  1  1 -1]
[ 1 -1 -1 -1  1  1 -1  1]
[ 1 -1 -1  1  1 -1  1 -1]
[ 1 -1  1  1 -1  1 -1 -1]
[ 1  1  1 -1  1 -1 -1 -1]

In [4]:
# La primera matriz con 6 columnas
M = Matrix([
    [-I, -I, I, -I, -1, -1],
    [1, -1, -1, -1, I, I],
    [1, 1, -1, 1, I, I],
    [I, -I, -I, -I, 1, 1],
    [-1, 1, -1, -1, -I, I],
    [I, I, I, -I, 1, -1],
    [I, -I, I, I, 1, -1],
    [1, 1, 1, -1, I, -I]
]) / sqrt(8)
M * sqrt(8)

[-I -I  I -I -1 -1]
[ 1 -1 -1 -1  I  I]
[ 1  1 -1  1  I  I]
[ I -I -I -I  1  1]
[-1  1 -1 -1 -I  I]
[ I  I  I -I  1 -1]
[ I -I  I  I  1 -1]
[ 1  1  1 -1  I -I]

In [5]:
a, b, c, d, e, f, g, h = SR.var('a b c d e f g h') # variables simbólicas

Si $M_i$ es el $i$-ésima vector columna de $M$, entonces podemos calcular todos los $\langle M_i|v \rangle$ donde $v$ es el vector $v = \begin{pmatrix}a & b & c & d & e & f & g & h\end{pmatrix}^\top$, haciendo la multiplicación $M^* v$. Igualando el vector a cero obtenemos un sistema de 8 variables y 6 ecuaciones:

In [6]:
eqns = list(M.conjugate_transpose() * vector([a,b,c,d,e,f,g,h]))
eqns = [eqn == 0 for eqn in eqns]
eqns

[1/4*I*sqrt(2)*a + 1/4*sqrt(2)*b + 1/4*sqrt(2)*c - 1/4*I*sqrt(2)*d - 1/4*sqrt(2)*e - 1/4*I*sqrt(2)*f - 1/4*I*sqrt(2)*g + 1/4*sqrt(2)*h == 0,
 1/4*I*sqrt(2)*a - 1/4*sqrt(2)*b + 1/4*sqrt(2)*c + 1/4*I*sqrt(2)*d + 1/4*sqrt(2)*e - 1/4*I*sqrt(2)*f + 1/4*I*sqrt(2)*g + 1/4*sqrt(2)*h == 0,
 -1/4*I*sqrt(2)*a - 1/4*sqrt(2)*b - 1/4*sqrt(2)*c + 1/4*I*sqrt(2)*d - 1/4*sqrt(2)*e - 1/4*I*sqrt(2)*f - 1/4*I*sqrt(2)*g + 1/4*sqrt(2)*h == 0,
 1/4*I*sqrt(2)*a - 1/4*sqrt(2)*b + 1/4*sqrt(2)*c + 1/4*I*sqrt(2)*d - 1/4*sqrt(2)*e + 1/4*I*sqrt(2)*f - 1/4*I*sqrt(2)*g - 1/4*sqrt(2)*h == 0,
 -1/4*sqrt(2)*a - 1/4*I*sqrt(2)*b - 1/4*I*sqrt(2)*c + 1/4*sqrt(2)*d + 1/4*I*sqrt(2)*e + 1/4*sqrt(2)*f + 1/4*sqrt(2)*g - 1/4*I*sqrt(2)*h == 0,
 -1/4*sqrt(2)*a - 1/4*I*sqrt(2)*b - 1/4*I*sqrt(2)*c + 1/4*sqrt(2)*d - 1/4*I*sqrt(2)*e - 1/4*sqrt(2)*f - 1/4*sqrt(2)*g + 1/4*I*sqrt(2)*h == 0]

Utilizamos el solver de SageMath y obtenemos un *supuesta* única solución.

In [7]:
sol = solve(eqns, [a,b,c,d,e,f,g,h], solution_dict=True, to_poly_solve='force')
sol

[{a: r2, b: r1*r2, c: -r1*r2, d: r2, e: r1^2, f: r1, g: -r1, h: r1^2}]

In [8]:
r1, r2 = SR.var('r1 r2')

In [9]:
v = vector(sol[0].values())
v

(r2, r1*r2, -r1*r2, r2, r1^2, r1, -r1, r1^2)

No se que condiciones espera SageMath porque la forma de la solución no es suficiente al parecer para que el vector sea ortonormal a los 6 vectores:

In [10]:
u = (M.conjugate_transpose() * v)
u

(0, 1/2*sqrt(2)*r1^2 - 1/2*sqrt(2)*r1*r2 - 1/2*I*sqrt(2)*r1 + 1/2*I*sqrt(2)*r2, 0, -1/2*sqrt(2)*r1^2 - 1/2*sqrt(2)*r1*r2 + 1/2*I*sqrt(2)*r1 + 1/2*I*sqrt(2)*r2, 0, 0)

Obtenemos dos componentes del vector que no son identiticamente igual a cero, resolvemos éste sistem $2 \times 2$:

In [11]:
solve([list(u)[1] == 0, list(u)[3] == 0], [r1, r2])

[[r1 == I, r2 == r3], [r1 == 0, r2 == 0]]

Al parecer obtenemos un solo grado de libertad (ignorando la solución trivial), con $r_1 = i$, el vector $v$ sí es ortonormal a los 6 vectores:

In [12]:
u.subs(r1=I)

(0, 0, 0, 0, 0, 0)

In [13]:
v.subs(r1=I)

(r2, I*r2, -I*r2, r2, -1, I, -I, -1)

Lo intenté con Mathematica y obtengo un vector con dos grados de libertad: $\begin{pmatrix} a & ia & -ia & a & h & -ih & ih & h\end{pmatrix}^\top$. Notemos que es igual al que obtengo con SageMath pero con $h = -1$. Probando con éste vector más general efectivamente verificamos que es ortonormal:

In [14]:
u = vector([a, I*a, -I*a, a, b, -I*b, I*b, b]) 
M.conjugate_transpose() * u

(0, 0, 0, 0, 0, 0)

El vector $u$ es ortorgonal, la condición de normalidad no agrega información:

In [15]:
M.conjugate_transpose() * (u / norm(u))

(0, 0, 0, 0, 0, 0)

In [16]:
u = u / norm(u)
norm(u).simplify_full()

1

Entonces tenemos un vector ortonormal $u$ y deseamos que sea mutuamente insesgado con *todos* los vectores de todas las bases. Consideremos la primera base:

In [17]:
Pf1 = F * diagonal_matrix([1,1,I,1,I,I,I,1]) * F.conjugate_transpose()
Pf1 * 2

[ I + 1 -I + 1      0      0      0      0      0      0]
[-I + 1  I + 1      0      0      0      0      0      0]
[     0      0  I + 1      0 -I + 1      0      0      0]
[     0      0      0  I + 1      0      0      0 -I + 1]
[     0      0 -I + 1      0  I + 1      0      0      0]
[     0      0      0      0      0  I + 1 -I + 1      0]
[     0      0      0      0      0 -I + 1  I + 1      0]
[     0      0      0 -I + 1      0      0      0  I + 1]

Igualamos el cuadrado del valor absoluto de cada componente de $P_{f_1}^* u$ a $1/8$ para fijar la condición, pero nos encontramos con un problema:

In [18]:
eqns = Pf1.conjugate_transpose() * u
# eqns = [abs(eqn)**2 == 1/8 for eqn in eqns]
eqns.simplify_full()

(0, (1/2*I + 1/2)*a/sqrt(a^2 + b^2), 1/4*(-(I + 1)*a + (I + 1)*b)/sqrt(a^2 + b^2), 1/4*(-(I - 1)*a + (I + 1)*b)/sqrt(a^2 + b^2), 1/4*(-(I - 1)*a - (I - 1)*b)/sqrt(a^2 + b^2), -1/2*b/sqrt(a^2 + b^2), 1/2*b/sqrt(a^2 + b^2), 1/4*((I + 1)*a - (I - 1)*b)/sqrt(a^2 + b^2))

El vector $u$ es ortonormal al primer vector de la matriz $P_{f_1}$, por lo tanto independientemente de los valores de $a$ y de $b$, el vector $u$ no puede ser mutuamente insesgado a la primera base. Si la solución que obtuve (con SageMath y con Mathematica) es la *única* solución para obtener un vector ortonormal a los de $M$, entonces no hay manera de completar la base.

In [24]:
solve(abs(b/(2*sqrt(a^2+b^2)))**2 == 1/8, b)

[b == -a, b == a]

In [34]:
eqns.subs(b=-a).subs(a=I).apply_map(lambda t: abs(t)**2).simplify_full()

(0, 1/4, 1/4, 1/8, 0, 1/8, 1/8, 1/8)